# Init

This notebook approximately follows the structure and adopts the notation of chapter 3 of [Burger & Burge: Digital Image Processing][1]

[1]: https://link.springer.com/book/10.1007/978-1-4471-6684-9

In [ ]:
from typing import Callable, Union

import cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy
import skimage

In [ ]:
sns.set()

In [ ]:
rgb = cv2.imread('data/rentgen.bmp')[..., ::-1]
rgb.dtype, rgb.shape, rgb.min(), rgb.max()

In [ ]:
gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
gray.dtype, gray.shape, gray.min(), gray.max()

In [ ]:
def imshow_pair(
        horizontal: bool = True,
        cmap: str = 'gray',
        vmin: Union[int, float] = 0,
        vmax: Union[int, float] = 255,
        grid: bool = False,
        **images
):
    nrows, ncols = (1, 2) if horizontal else (2, 1)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, layout='constrained')
    for ax, (name, img) in zip(axes.flat, images.items()):
        im = ax.imshow(img, cmap=cmap, vmin=vmin, vmax=vmax)
        ax.grid(grid)
        ax.set_title(name)
    return fig, axes

In [ ]:
imshow_pair(rgb=rgb, gray=gray);

# Brightness transformations

In general, brightness transformation is
$$
I'(x, y) = f(I(x, y), x, y)
$$
where
- $I(x, y)$ is the original image,
- $f$ is a transformation function,
- $I'(x, y)$ is the newly produced transformed image.

Notice that the output value $a' = I'(x, y)$ at position $(x, y)$ only depends on input value $a = I(x, y)$ at the same coordinates $(x, y)$. This means that
**brightness transformations do not change size, geometry or local structure of the image**.

# Monadic operations

A special category of brightness transformations are monadic (unary, single input) operations, which are independent of the position $(x, y)$ in the image.
$$
a' = f(I(x, y)) = f(a)
$$
where
- $a = I(x, y)$ is the original brightness,
- $a' = I'(x, y)$ the transformed brightness.

## Change of brightness

**Manipulating brightness has an additive effect:**
$$
f(a) = a + \beta
$$
For example, to increase the brightness by 10, we would do $f(a) = a + 10$

In [ ]:
def change_brightness(img: np.ndarray, beta: float) -> np.ndarray:
    height, width = img.shape
    output = np.zeros_like(img)
    for y in range(height):
        for x in range(width):
            b = img[y, x] + beta
            if b > 255:
                b = 255
            output[y, x] = b
    return output

In [ ]:
gray_brt = change_brightness(gray, 50)
gray_brt.dtype, gray_brt.shape, gray_brt.min(), gray_brt.max()

In [ ]:
imshow_pair(gray=gray, gray_brt=gray_brt);

## Change of contrast

**Manipulating contrast has a multiplicative effect:**
$$
f(a) = \alpha \cdot a
$$
For example, to increase the contrast by 50 %, we would do $f(a) = 1.5\cdot a$

In [ ]:
def change_contrast(img: np.ndarray, alpha: float) -> np.ndarray:
    height, width = img.shape
    output = np.zeros_like(img)
    for y in range(height):
        for x in range(width):
            output[y, x] = img[y, x] * alpha  # FIXME: a problem here
    return output

In [ ]:
gray_con = change_contrast(gray, 1.5)
gray_con.dtype, gray_con.shape, gray_con.min(), gray_con.max()

In [ ]:
imshow_pair(gray=gray, gray_con=gray_con);

# Inverting an image (image negative)

The transformation function is
$$f(a) = a_\textrm{max} - a$$
For `uint8`, $a_\textrm{max} = 255$.

In [ ]:
def image_negative_slow(img: np.ndarray, v_max: int = 255) -> np.ndarray:
    height, width = img.shape
    output = np.zeros_like(img)
    for y in range(height):
        for x in range(width):
            output[y, x] = v_max - img[y, x]
    return output

In [ ]:
gray_neg = image_negative_slow(gray)
gray_neg.dtype, gray_neg.shape, gray_neg.min(), gray_neg.max()

In [ ]:
imshow_pair(gray=gray, gray_neg=gray_neg);

## Fast vectorized implementation

In [ ]:
def image_negative(img: np.ndarray, v_max: int = 255) -> np.ndarray:
    return v_max - img

In [ ]:
gray_neg = image_negative(gray)
gray_neg.dtype, gray_neg.shape, gray_neg.min(), gray_neg.max()

In [ ]:
imshow_pair(gray=gray, gray_neg=gray_neg);

In [ ]:
%timeit image_negative_slow(gray)

In [ ]:
%timeit image_negative(gray)

# Thresholding

Thresholding is a brightness transformation function that separates pixels by their value into two classes, for example background and foreground.
$$
f(a; t) = \begin{cases}
    a_\textrm{min} & \textrm{if} \; a \lt t \\
    a_\textrm{max} & \textrm{if} \; a \ge t
\end{cases}
$$
where
- $t$ is some threshold value.

Usually, we want to *binarize* the image and therefore choose $a_\textrm{min} = 0$ and $a_\textrm{max} = 1$ (or sometimes 0 and 255 in case of `uint8` images).

In [ ]:
def threshold_image(img: np.ndarray, threshold: int, v_max: int = 255) -> np.ndarray:
    output = np.zeros_like(img)
    output[img >= threshold] = v_max
    return output

In [ ]:
gray_thr = threshold_image(gray, 125)
gray_thr.dtype, gray_thr.shape, gray_thr.min(), gray_thr.max()

In [ ]:
imshow_pair(gray=gray, gray_thr=gray_thr);

# Automatic contrast adjustment (enhancement)

Let's first look at the image histogram:

In [ ]:
h_bins = np.arange(257)
h_gray, _ = np.histogram(gray, bins=h_bins)

In [ ]:
plt.bar(h_bins[:-1], h_gray, width=1., edgecolor='none');

If the image only covers a small range of intensities, it may appear as low contrast. In order to increase the contrast, we need to fully utilize the entire range of possible intensities like in the following picture.

<img src="figures/brightness_transformations-auto_contrast_adjust.png" alt="Drawing" style="width: 6in;"/>

Let's say the smallest and the highest values in our image $I$ are $a_\textrm{lo} = \min(I)$ and $a_\textrm{hi} = \max(I)$, respectively.

In [ ]:
gray.min(), gray.max()

In order to cover some specified brightness range, we will map the smallest value $a_\textrm{lo}$ to zero (let's call that zero $a_\textrm{min}$) and the highest value $a_\textrm{hi}$ to 255 ($=a_\textrm{max}$) like so
$$
f(a) = \frac{a_\textrm{max} - a_\textrm{min}}{a_\textrm{hi} - a_\textrm{lo}} \cdot (a - a_\textrm{lo}) + a_\textrm{min}
$$
which for an 8-bit with $a_\textrm{min} = 0$ and $a_\textrm{max} = 255$ simplifies to
$$
f(a) = \frac{255}{a_\textrm{hi} - a_\textrm{lo}} \cdot (a - a_\textrm{lo})
$$

In [ ]:
def normalize_minmax_uint8(img: np.ndarray) -> np.ndarray:
    a_lo, a_hi = img.min(), img.max()
    output = 255. / (a_hi - a_lo) * (img - a_lo)
    return output.astype(np.uint8)

In [ ]:
gray_nrm = normalize_minmax_uint8(gray)
gray_nrm.dtype, gray_nrm.shape, gray_nrm.min(), gray_nrm.max()

In [ ]:
imshow_pair(gray=gray, gray_nrm=gray_nrm);

# Automatic contrast adjustment for noisy images

Simple min-max normalization will fail when there are outlier values in the input image. Let's corrupt our image with artificial salt & pepper noise to illustrate the point. Each pixel will be randomly set to 0 or 255 with some probability (`amount` in the following code).

In [ ]:
gray_snp = skimage.util.random_noise(gray, mode='s&p', amount=0.01)  # this also converts to float and 0..1 range
gray_snp = skimage.util.img_as_ubyte(gray_snp)  # convert back to uint8 and 0..255
gray_snp.dtype, gray_snp.shape, gray_snp.min(), gray_snp.max()

In [ ]:
imshow_pair(gray=gray, gray_snp=gray_snp);

In [ ]:
plt.hist(gray_snp.ravel(), bins=h_bins, width=1., edgecolor='none');

In [ ]:
gray_snp_nrm = normalize_minmax_uint8(gray_snp)
gray_snp_nrm.dtype, gray_snp_nrm.shape, gray_snp_nrm.min(), gray_snp_nrm.max()

In [ ]:
imshow_pair(gray_snp=gray_snp, gray_snp_nrm=gray_snp_nrm);

Obviously, because the original brightness values already covered the entire `uint8` range 0..255 due to the added noise, there was nothing to "stretch". In order to remedy this, we need to estimate the original brightness range *before* the noise corruption. One idea is to assume the type of noise (salt & pepper) and the maximum amount (5 % in our case). Then, insted of hard max, we can set $a_\textrm{hi}$ to be a value such that 95 % of pixels in the image are smaller, and similarly set the $a_\textrm{lo}$ as well. See the following illustration, where the percentage of pixels with the few lowest and highest values are denoted as $p_\textrm{lo}$ and $p_\textrm{hi}$, respectively.

<img src="figures/brightness_transformations-percentile_contrast_adjustment.png" alt="Drawing" style="width: 6in;"/>

Both values $a_\textrm{lo}$, $a_\textrm{hi}$ can be calculated using the `np.percentile` function. However, will do so manually using a cumulative histogram, since the `np.percentile` relies on cumulative distribution function anyway and it will be useful later for implementation of histogram equalization.

## Cumulative histogram

If we have a histogram $h$ of our image $I$, the cumulative histogram $H$ is defined as
$$
H(a) = \sum_{i=0}^i h(i) \quad \textrm{for} \; 0 \le a \lt K
$$
where
- $K$ is the number of brightness levels, e.g. $K = 256$ for `uint8` images.

It holds that
- $a$-th value in $H$ is the sum of all values in $h$ up to the $a$-th position,
- the last element $H(K-1)$ is equal to the number of pixels in $I$. If $M$ and $N$ are the width and height of $I$, respectively, then $H(K-1) = M \cdot N$.

In [ ]:
def cumsum_naive(h: np.ndarray) -> np.ndarray:
    H = np.zeros_like(h)
    for a in range(len(h)):
        for i in range(a+1):
            H[a] += h[i]
    return H

In [ ]:
def cumsum_recursive(h: np.ndarray) -> np.ndarray:
    H = np.zeros_like(h)
    H[0] = h[0]
    for i in range(2, len(h)):
        H[i] = H[i - 1] + h[i]
    return H

In [ ]:
np.all(cumsum_naive(h_gray) == cumsum_recursive(h_gray))

In [ ]:
%timeit cumsum_naive(h_gray)

In [ ]:
%timeit cumsum_recursive(h_gray)

In [ ]:
np.all(np.cumsum(h_gray) == cumsum_recursive(h_gray))

In [ ]:
%timeit np.cumsum(h_gray)

In [ ]:
h_gray_cum = np.cumsum(h_gray)
h_gray_cum.dtype, h_gray_cum.shape, h_gray_cum.min(), h_gray_cum.max()

In [ ]:
plt.bar(h_bins[:256], h_gray, width=1, alpha=0.2)
plt.xlabel('$a$')
plt.ylabel('$h(a)$')
plt.gca().twinx()
plt.plot(h_gray_cum)
plt.ylabel('$H(a)$')
plt.ylim(-1000., None)

## Frequencies and probabilities

The value in each histogram bin describes the observed frequency of the corresponding intensity value, i.e. we may treat the histogram as a discrete *frequency distribution*. For a given image $I$ of size $M \times N$, the sum of all histogram entries is equal to the number of pixels
$$
\sum_a{h(a)} = M \cdot N
$$

The associated normalized histogram
$$
p(a) = \frac{h(a)}{M \cdot N}
$$
can be interpreted as the probability mass function (pmf), where $p(a)$ is the probability for the occurence of the pixel value $a$. As a probability distribution, it satisfies
$$
\sum_a{p(a)} = 1
$$
The statistical counterpart to the cumulative histogram $H$ is the *cumulative distribution function*
$$
\textrm{cdf}(a) = \sum_{i=0}^a{p(i)} = \frac{H(a)}{M \cdot N}
$$
Each value $\textrm{cdf}(a)$ tells us the percentile of $a$. If, for example, $\textrm{cdf}(10) = 0.07$, it means that 7 % of pixels in the image $I$ have a value smaller or equal to $10$. The last value $\textrm{cdf}(K-1)$ (typically $K=256$ for `uint8` images) is always equal to one.

In [ ]:
def pmf(h: np.ndarray) -> np.ndarray:
    return h / h.sum()

In [ ]:
p_gray = pmf(h_gray)
p_gray.shape, p_gray.dtype, p_gray.min(), p_gray.max()

In [ ]:
def cdf(h: np.ndarray) -> np.ndarray:
    p_cum = np.cumsum(h)
    return p_cum / p_cum[-1]

In [ ]:
p_gray_cum = cdf(h_gray)
p_gray_cum.shape, p_gray_cum.dtype, p_gray_cum.min(), p_gray_cum.max()

In [ ]:
with sns.axes_style('darkgrid'):
    plt.bar(h_bins[:256], p_gray, width=1., alpha=0.2)
    plt.xlabel('$a$')
    plt.ylabel('$p(a)$')
    plt.gca().twinx()
    plt.plot(p_gray_cum)
    plt.ylabel('$\\text{cdf}(a)$')
    plt.ylim(-0.01, None)

## Normalization range limits $a_\textrm{lo}$ and $a_\textrm{hi}$ as percentiles

Using cumulative distribution function $\textrm{cdf}$, we can easily find brightness values $a_\textrm{lo}$ and $a_\textrm{hi}$, which correspond to the smallest and largest few percent of values in $I$, respectively. The value $a_\textrm{lo}$ will be the first entry in the $\textrm{cdf}$ which is larger than e.g. 5 %. If we denote the percentage as $p_\textrm{lo}$, then
$$
a_\textrm{lo} = \min\left\{a | \textrm{cdf}(a) \ge p_\textrm{lo}\right\}
$$

In [ ]:
a_lo = next(a for a in range(len(h_gray_cum)) if p_gray_cum[a] >= 0.05)  # p_lo = 0.05
a_lo

Similarly, $a_\textrm{hi}$ will be the last $a$ for which $\textrm{cdf}(a) \le 0.95$, i.e.
$$
a_\textrm{hi} = \max\left\{a | \textrm{cdf}(a) \le (1 - p_\textrm{hi})\right\}
$$

In [ ]:
a_hi = next(a for a in reversed(range(len(h_gray_cum))) if p_gray_cum[a] <= 0.95)  # p_hi = 0.05
a_hi

Let's put the entire percentile-based contrast adjustment into single function.

In [ ]:
def normalize_percentile_uint8(img: np.ndarray, percentage: float = 0.05) -> np.ndarray:
    h, _ = np.histogram(img.ravel(), bins=np.arange(257))
    p_cum = cdf(h)
    a_lo = next(a for a in range(len(p_cum)) if p_cum[a] > percentage)
    a_hi = next(a for a in reversed(range(len(p_cum))) if p_cum[a] < 1. - percentage)
    output = 255. / (a_hi - a_lo) * (img.astype(float) - a_lo)
    return output.clip(min=0, max=255).astype(np.uint8)

In [ ]:
gray_snp_nrp = normalize_percentile_uint8(gray_snp, percentage=0.05)  # try lowering percentile here
gray_snp_nrp.dtype, gray_snp_nrp.shape, gray_snp_nrp.min(), gray_snp_nrp.max()

In [ ]:
imshow_pair(gray_snp=gray_snp, gray_snp_nrp=gray_snp_nrp);

# Brightness transformations as an application of the function $f$

Notice that in all of the examples above, the procedure for transforming the input image using a monadic transformation function $f$ was always the same. The only thing that varied was the function $f$. Therefore, we may create a general function `remap_values` that takes in two inputs: image and $f$.

In [ ]:
def remap_values(img: np.ndarray, f: Callable[[float], float]) -> np.ndarray:
    height, width = img.shape
    output = np.zeros_like(img)
    for y in range(height):
        for x in range(width):
            output[y, x] = f(img[y, x])
    return output

## Inverting an image (image negative)

We'll look at image negative again. However this time, we will do it using the `remap_values` function. Remember that the image negative transformation function is
$$f(a) = a_\textrm{max} - a$$
For `uint8`, $a_\textrm{max} = 255$. Let's create $f$ as a Python function.

In [ ]:
def f_neg(a: float) -> float:
    return 255 - a

In [ ]:
gray_neg = remap_values(gray, f_neg)
gray_neg.dtype, gray_neg.shape, gray_neg.min(), gray_neg.max()

In [ ]:
imshow_pair(gray=gray, gray_neg=gray_neg);

## Pre-computing $f$ as a lookup table (LUT)

When we are working with discrete images with finite number of distinct pixel values, the transformation function $f(a)$ will be a discrete function with a finite number of distinct values of $a$. Typically, for `uint8`, $a = 0,\ldots,255$. We may then pre-compute the output values $f(a)$ so that $f$ need not to be re-evaluated for each pixel inside the for loop of `transform_values`. We'll create another function `transform_values_uint8`, in which the second argument will be a vector of size 256 with pre-computed values of $f$ instead of $f$ itself.

In [ ]:
def remap_values_uint8(img: np.ndarray, f_lut: np.ndarray) -> np.ndarray:
    height, width = img.shape
    output = np.zeros_like(img)
    for y in range(height):
        for x in range(width):
            output[y, x] = f_lut[img[y, x]]
    return output

In [ ]:
def create_image_negative_lut() -> np.ndarray:
    return 255 - np.arange(256, dtype=np.uint8)

In [ ]:
lut_neg = create_image_negative_lut()
lut_neg.shape, lut_neg.dtype, lut_neg.min(), lut_neg.max()

In [ ]:
def plot_lut(lut: np.ndarray):
    plt.plot(lut);
    plt.xlabel('$a$')
    plt.ylabel('$a\'$')

In [ ]:
plot_lut(lut_neg)

In [ ]:
gray_neg = remap_values_uint8(gray, lut_neg)
gray_neg.dtype, gray_neg.shape, gray_neg.min(), gray_neg.max()

In [ ]:
imshow_pair(gray=gray, gray_neg=gray_neg);

We can verify that the LUT approach is significantly faster.

In [ ]:
%timeit remap_values(gray, f_neg)

In [ ]:
%timeit remap_values_uint8(gray, lut_neg)

Moreover, we can take advantage of highly optimized Numpy indexing and speed up the transformation even further.

In [ ]:
gray_neg = lut_neg[gray]
gray_neg.dtype, gray_neg.shape, gray_neg.min(), gray_neg.max()

In [ ]:
imshow_pair(gray=gray, gray_neg=gray_neg);

In [ ]:
%timeit lut_neg[gray]

## Thresholding via LUT

In [ ]:
def create_thresholding_lut(threshold: int) -> np.ndarray:
    lut = np.zeros(256, dtype=np.uint8)
    lut[threshold:] = 255
    return lut

In [ ]:
lut_thr = create_thresholding_lut(125)

In [ ]:
plot_lut(lut_thr)

In [ ]:
gray_thr = lut_thr[gray]
gray_thr.dtype, gray_thr.shape, gray_thr.min(), gray_thr.max()

In [ ]:
imshow_pair(gray=gray, gray_thr=gray_thr);

## Contrast adjustment via LUT

In [ ]:
def create_contrast_adjustment_lut(a_lo: int, a_hi: int, a_min: int, a_max: int) -> np.ndarray:
    lut = (a_max - a_min) / (a_hi - a_lo) * (np.arange(256, dtype=float) - a_lo) + a_min
    return lut.clip(0, 255).astype(np.uint8)

In [ ]:
lut_adj = create_contrast_adjustment_lut(a_lo, a_hi, 0, 255)

In [ ]:
plot_lut(lut_adj)

In [ ]:
gray_nrm = lut_adj[gray]
gray_nrm.shape, gray_nrm.dtype, gray_nrm.min(), gray_nrm.max()

In [ ]:
imshow_pair(gray=gray, gray_nrm=gray_nrm);

# Histogram equalization

The goal of histogram equalization is to find a value mapping such that the histogram of the remmaped image approximates a uniform distribution. The resulting image will appear as high contrast, since all possible rightness values will be utilized in roughly the same amount.

The principle is illustrated in the following figure. We are trying to find a mapping that shifts the histogam such that the resulting cumulative histogram resembles a linear ramp, i.e. like an ideal uniformly distributed image would.

<img src="figures/brightness_transformations-cumulative_histogram.png" alt="Drawing" style="width: 6in;"/>

Basically, the idea is to stretch all dense clusters of similar values to be farther apart, in order to make them visually more distinct. This is similar to e.g. automatic contrast adjustment, but equalization remaps the values non-linearly based on local histogram density

The desired operation is obtained from the cumulative histogram $H$ of the original image as
$$
f(a) = \left\lfloor H(a) \cdot \frac{K-1}{M \cdot N} \right\rfloor = \left\lfloor \textrm{cdf}(a) \cdot (K-1) \right\rfloor
$$

In [ ]:
def create_histogram_equalization_lut(h: np.ndarray) -> np.ndarray:
    h_cum = cdf(h)
    lut = np.floor(h_cum * 255)
    return lut.astype(np.uint8)

In [ ]:
# let's try histogram equalization with noisy image to demonstrate its robustness
lut_equ = create_histogram_equalization_lut(np.histogram(gray_snp, bins=np.arange(257))[0])

In [ ]:
plot_lut(lut_equ)

In [ ]:
gray_snp_equ = lut_equ[gray_snp]
gray_snp_equ.shape, gray_snp_equ.dtype, gray_snp_equ.min(), gray_snp_equ.max()

In [ ]:
imshow_pair(gray_snp=gray_snp, gray_snp_equ=gray_snp_equ);

In [ ]:
with sns.axes_style('darkgrid'):
    plt.figure(figsize=(6.4, 2.4))
    plt.subplot(1, 2, 1)
    plt.bar(h_bins[:256], h_gray, width=1., edgecolor='none')
    plt.subplot(1, 2, 2)
    plt.bar(h_bins[:256], np.histogram(gray_snp_equ, bins=h_bins)[0], width=1., edgecolor='none')
    plt.tight_layout()

# Histogram matching

We also may want to remap one image in such a way that the resulting histogram will aproximate a histogram of another image.

<img src="figures/brightness_transformations-histogram_matching.png" alt="Drawing" style="width: 6in;"/>

In [ ]:
def create_histogram_match_lut(h_input: np.ndarray, h_ref: np.ndarray) -> np.ndarray:
    cdf_input = cdf(h_input)
    cdf_ref = cdf(h_ref)
    lut = np.zeros(len(h_input), dtype=np.uint8)
    for a in range(len(h_input)):
        try:
            lut[a] = next(i for i in reversed(range(len(h_input))) if cdf_input[a] > cdf_ref[i])
        except StopIteration:
            lut[a] = 0
    return lut

In [ ]:
h_refs = [
    scipy.stats.beta(alpha, beta).pdf(np.linspace(0.01, 0.99, 256))
    for alpha, beta in ((0.5, 0.5), (5., 1.), (1., 3.), (2., 2.))
]

In [ ]:
with sns.axes_style('darkgrid'):
    for a, h_ref in enumerate(h_refs):
        plt.subplot(2, 2, a + 1)
        plt.bar(h_bins[:256], h_ref, width=1, edgecolor='none');
    plt.tight_layout()

In [ ]:
lut_mchs = [
    create_histogram_match_lut(h_gray, h_ref)
    for h_ref in h_refs
]

In [ ]:
with sns.axes_style('darkgrid'):
    for a, lut_mch in enumerate(lut_mchs):
        plt.subplot(2, 2, a + 1)
        plt.plot(lut_mch);
    plt.tight_layout()

In [ ]:
gray_mchs = [lut_mch[gray] for lut_mch in lut_mchs]

In [ ]:
with sns.axes_style('darkgrid'):
    plt.figure(figsize=(2. * plt.rcParams['figure.figsize'][0], 2. * plt.rcParams['figure.figsize'][1]))
    for a, (h_ref, lut_mch, gray_mch) in enumerate(zip(h_refs, lut_mchs, gray_mchs)):
        plt.subplot(4, 4, 4 * a + 1)
        plt.bar(h_bins[:256], h_ref, edgecolor='none', width=1.)  # 1st column: target histogram
        plt.subplot(4, 4, 4 * a + 2)
        plt.plot(lut_mch);  # 2nd column: mapping function (LUT)
        plt.subplot(4, 4, 4 * a + 3)
        plt.imshow(gray_mch, cmap='gray', vmin=0, vmax=255);  # 3rd column: remapped image
        plt.grid(False)
        plt.subplot(4, 4, 4 * a + 4)
        plt.bar(h_bins[:256], np.histogram(gray_mch, bins=h_bins)[0], edgecolor='none', width=1.)  # 4th column: histogram of the remapped image
    plt.tight_layout();

# Gamma correction

Usually, the relationship between physical signal amplitude such as light intensity emitted by an LCD screen and its input voltage, or vice versa, charge accumulated on a camera chip and the converted intensity, is non-linear. Perhaps surprisingly, a lot of these relationships can be modeled as an exponential function of the form
$$
f(a) = a^\gamma
$$
where
- $\gamma \in \mathbb{R}$ is a parameter called gamma value.

Depending on the gamma value, the function can look like the following figure.

<img src="figures/brightness_transformations-gamma_function.png" alt="Drawing" style="width: 6in;"/>

We may use $f(a)$ in a process called *gamma correction*, in which we try to linearize the input-output relationship in order to standardize stored values across different devices. See the following figure.

<img src="figures/brightness_transformations-gamma_correction.png" alt="Drawing" style="width: 6in;"/>

# Comparison of brightness transformations

In [ ]:
with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(nrows=5, ncols=2, constrained_layout=True, figsize=(plt.rcParams['figure.figsize'][0], 2.5*plt.rcParams['figure.figsize'][1]))
    for axr, lut in zip(axes, [np.arange(256), lut_adj, lut_equ, lut_thr, lut_neg]):
        axr[0].plot(lut);
        axr[1].imshow(lut[gray], cmap='gray', vmin=0, vmax=255)
        axr[1].grid()

# Brightness transformations dependent on position

## Vignette effect

In [ ]:
rgb = cv2.imread('data/sunflowers.png')[..., ::-1]

In [ ]:
with sns.axes_style('dark'):
    plt.imshow(rgb);

Create the vignette effect by calculating (axis normalized) distance from the center:

In [ ]:
# vignette
x, y = np.meshgrid(np.arange(rgb.shape[1]), np.arange(rgb.shape[0]))
xc, yc = rgb.shape[1] / 2, rgb.shape[0] / 2
dx2 = (x - xc) ** 2 / rgb.shape[1] ** 2
dy2 = (y - yc) ** 2 / rgb.shape[0] ** 2
vig_dist = np.clip(1.2 - np.sqrt(dx2 + dy2), 0., 1.)

In [ ]:
with sns.axes_style('dark'):
    plt.imshow(vig_dist)
    plt.colorbar(shrink=0.5, aspect=10);

Alternatively, we can use 2D Gaussian distribution:

In [ ]:
x, y = np.meshgrid(np.arange(rgb.shape[1]), np.arange(rgb.shape[0]))
vig_gauss = np.clip(-1. + 2.15 * np.exp(-(x - xc) ** 2 / rgb.shape[1] ** 2 - (y - yc) ** 2 / rgb.shape[0] ** 2), 0., 1.)

In [ ]:
with sns.axes_style('dark'):
    plt.imshow(vig_gauss)
    plt.colorbar(shrink=0.5, aspect=10);

Multiply the input image `rgb` by the `vig_*` matrix:

In [ ]:
# channel by channel
rgb_vig = np.zeros(rgb.shape, dtype=rgb.dtype)
for c in range(rgb.shape[2]):
    rgb_vig[..., c] = vig_dist * rgb[..., c]
rgb_vig.dtype

In [ ]:
# broadcasting
rgb_vig = (vig_gauss[..., None] * rgb).astype(np.uint8)
rgb_vig.dtype, rgb_vig.shape

In [ ]:
imshow_pair(rgb=rgb, rgb_vig=rgb_vig, horizontal=False);